This example notebook shows the features of the different expectation operators that are available in sQUlearn.
Also, the computation of the derivatives is shown in the notebook.

In [1]:
# Import the expectation operator classes
from squlearn.expectation_operator import (
    SinglePauli,
    SummedPaulis,
    SingleProbability,
    SummedProbabilities,
    IsingHamiltonian,
    CustomExpectationOperator,
)

# Import class for derivatives of the expectation operator
from squlearn.expectation_operator.expectation_operator_derivatives import (
    ExpectationOperatorDerivatives,
)

# Import ParamterVector class from qiskit that is used for parameterized operators
from qiskit.circuit import ParameterVector

The expectation values of Pauli Matrices of single qubits can be evaluated utilizing the ``SinglePauli`` operator class.

In [2]:
# Operator for evaluating the expectation value in a 4 qubit system of the Z Pauli matrix of qubit 2
op = SinglePauli(num_qubits=4, qubit=2, op_str="Z")
print("Operator for measuring Z matrix of qubit 2:\n", op.get_pauli([]), "\n")

# The other Pauli matrices are possible es well:
op = SinglePauli(num_qubits=4, qubit=2, op_str="X")
print("Operator for measuring X:\n", op.get_pauli([]), "\n")

op = SinglePauli(num_qubits=4, qubit=2, op_str="Y")
print("Operator for measuring Y:\n", op.get_pauli([]), "\n")

op = SinglePauli(num_qubits=4, qubit=2, op_str="I")
print("Operator for measuring I:\n", op.get_pauli([]), "\n")

op = SinglePauli(num_qubits=4, qubit=2, parameterized=True)
print(
    "Operators with a trainable parameters are possible as well:\n",
    op.get_pauli(ParameterVector("p", op.num_parameters)),
    "\n",
)

Operator for measuring Z matrix of qubit 2:
 IZII 

Operator for measuring X:
 IXII 

Operator for measuring Y:
 IYII 

Operator for measuring I:
 IIII 

Operators with a trainable parameters are possible as well:
 1.0*p[0] * IZII 



Sums of Pauli operators are also possible with the ``SummedPaulis`` class:

$\hat{H} = a \hat{I} + \sum\limits_i b_i \hat{Z}_i$

 Here, the identity is used for a constant offset of the expectation value.
It is possible to either parameterize each Pauli matrix or to parameterize the full sum.


In [3]:
op = SummedPaulis(num_qubits=4, op_str="Z")
print(
    "Summation over Z operators for each qubit:\n",
    op.get_pauli(ParameterVector("p", op.num_parameters)),
    "\n",
)

op = SummedPaulis(num_qubits=4, op_str="Z", full_sum=False)
print(
    "It is also possible to move the parameter outside the sum:\n",
    op.get_pauli(ParameterVector("p", op.num_parameters)),
    "\n",
)

op = SummedPaulis(num_qubits=4, op_str=["Z", "Y"])
print(
    "A sum over selected Pauli operators are possible as well:\n",
    op.get_pauli(ParameterVector("p", op.num_parameters)),
    "\n",
)

Summation over Z operators for each qubit:
 SummedOp([
  1.0*p[0] * IIII,
  1.0*p[1] * IIIZ,
  1.0*p[2] * IIZI,
  1.0*p[3] * IZII,
  1.0*p[4] * ZIII
]) 

It is also possible to move the parameter outside the sum:
 SummedOp([
  1.0*p[0] * IIII,
  1.0*p[1] * IIIZ,
  1.0*p[1] * IIZI,
  1.0*p[1] * IZII,
  1.0*p[1] * ZIII
]) 

A sum over selected Pauli operators are possible as well:
 SummedOp([
  1.0*p[0] * IIII,
  1.0*p[1] * IIIZ,
  1.0*p[2] * IIZI,
  1.0*p[3] * IZII,
  1.0*p[4] * ZIII,
  1.0*p[5] * IIIY,
  1.0*p[6] * IIYI,
  1.0*p[7] * IYII,
  1.0*p[8] * YIII
]) 



Probabilities of measuring a certain state in a certain qubit can be obtained with the class ``SingleProbability``.
The probabilities are measured by using the following identity:

$ P_0^2 = \left\langle \Psi | 0 \right\rangle \left\langle 0 | \Psi \right\rangle $.

This is computed by the expectation values of the following operators:

$ \left| 0 \right\rangle \left\langle 0 \right| = 0.5(\hat{I}+\hat{Z})$ and  $ \left| 1 \right\rangle \left\langle 1 \right| = 0.5(\hat{I}-\hat{Z})$

In [4]:
# Example: |0><0| operator
op = SingleProbability(num_qubits=4, qubit=1)
print("Operator |0><0|:\n", op.get_pauli([]), "\n")

# Example: |1><1| operator
op = SingleProbability(num_qubits=4, qubit=1, one_state=True)
print("Operator |1><1|:\n", op.get_pauli([]), "\n")

# Example: |0><0| operator with parameters
op = SingleProbability(num_qubits=4, qubit=1, parameterized=True)
print("Parameterized Operator:\n", op.get_pauli(ParameterVector("p", op.num_parameters)), "\n")

Operator |0><0|:
 0.5 * IIII
+ 0.5 * IIZI 

Operator |1><1|:
 0.5 * IIII
- 0.5 * IIZI 

Parameterized Operator:
 1.0*p[0] * (
  0.5 * IIII
  + 0.5 * IIZI
) 



Furthermore, summing up the probabilities of multiple qubits is also possible with the ``SummedProbabilities`` operator.
Additionally a parameterized offset is achieved by the identity function.

In [5]:
# Summed |0><0| operator (note that the identity terms are condensed)
op = SummedProbabilities(num_qubits=4)
print("Summed |0><0| operator:\n", op.get_pauli(ParameterVector("p", op.num_parameters)), "\n")

# Summed |1><1| operator (note that the identity terms are condensed)
op = SummedProbabilities(num_qubits=4, one_state=True)
print("Summed |1><1| operator:\n", op.get_pauli(ParameterVector("p", op.num_parameters)), "\n")

# It is also possible to use a single parameter for the whole sum:
op = SummedProbabilities(num_qubits=4, full_sum=False)
print(
    "Summed |0><0| operator with a single parameter:\n",
    op.get_pauli(ParameterVector("p", op.num_parameters)),
    "\n",
)

Summed |0><0| operator:
 SummedOp([
  1.0*p[0] + 0.5*p[1] + 0.5*p[2] + 0.5*p[3] + 0.5*p[4] * IIII,
  0.5*p[1] * IIIZ,
  0.5*p[2] * IIZI,
  0.5*p[3] * IZII,
  0.5*p[4] * ZIII
]) 

Summed |1><1| operator:
 SummedOp([
  1.0*p[0] + 0.5*p[1] + 0.5*p[2] + 0.5*p[3] + 0.5*p[4] * IIII,
  -0.5*p[1] * IIIZ,
  -0.5*p[2] * IIZI,
  -0.5*p[3] * IZII,
  -0.5*p[4] * ZIII
]) 

Summed |0><0| operator with a single parameter:
 SummedOp([
  1.0*p[0] + 2.0*p[1] * IIII,
  0.5*p[1] * IIIZ,
  0.5*p[1] * IIZI,
  0.5*p[1] * IZII,
  0.5*p[1] * ZIII
]) 



There is also the class ``IsingHamiltonian`` for constructing Ising Hamiltonian kind operators:

$ \hat{H} = a\hat{I} + \sum\limits_i b_i \hat{Z}_i + \sum\limits_i c_i \hat{X}_i + \sum\limits_{i>j} d_{ij} \hat{Z}_i \hat{Z}_j $

The shape of the operator can be changed by the values of the input variables Z, X, and ZZ of the class:
 - ``'N'``: is removed from the Ising Hamiltonian
 - ``'S'`` is added with the trainable parameter outside the sum
 - ``'F'`` is added with the trainable parameter inside the sum


In [6]:
# Default Ising Hamiltonian:
op = IsingHamiltonian(num_qubits=4)
print("Default Ising Hamiltonian:\n", op.get_pauli(ParameterVector("p", op.num_parameters)), "\n")

op = IsingHamiltonian(num_qubits=4, Z="N", X=" F", ZZ="S")
print(
    "The shape of the operator can be adjusted by setting Z, X, and ZZ:\n",
    op.get_pauli(ParameterVector("p", op.num_parameters)),
    "\n",
)

Default Ising Hamiltonian:
 SummedOp([
  1.0*p[0] * IIII,
  1.0*p[1] * IIIZ,
  1.0*p[2] * IIZI,
  1.0*p[3] * IZII,
  1.0*p[4] * ZIII,
  1.0*p[5] * IIZZ,
  1.0*p[6] * IZIZ,
  1.0*p[7] * IZZI,
  1.0*p[8] * ZIIZ,
  1.0*p[9] * ZIZI,
  1.0*p[10] * ZZII
]) 

The shape of the operator can be adjusted by setting Z, X, and ZZ:
 SummedOp([
  1.0*p[0] * IIII,
  1.0*p[1] * IIZZ,
  1.0*p[1] * IZIZ,
  1.0*p[1] * IZZI,
  1.0*p[1] * ZIIZ,
  1.0*p[1] * ZIZI,
  1.0*p[1] * ZZII
]) 



Additionally it is possible to create custom Pauli based operators from an inputted string

In [7]:
# Example for a custom operator measuring Z in qubit 0, 1, and 3
op = CustomExpectationOperator(num_qubits=4, operator_string="XIYZ")
print("Custom operator:\n", op.get_pauli([]), "\n")

# Multiple operators that are summed can be combined bv a list/tuple:
op = CustomExpectationOperator(num_qubits=4, operator_string=["ZIZZ", "XIXI"])
print("Custom operator with multiple operators:\n", op.get_pauli([]), "\n")

# It is also possible to add trainable parameters:
op = CustomExpectationOperator(num_qubits=4, operator_string=["ZIZZ", "XIXI"], parameterized=True)
print(
    "Custom operator with multiple operators:\n",
    op.get_pauli(ParameterVector("p", op.num_parameters)),
    "\n",
)

Custom operator:
 XIYZ 

Custom operator with multiple operators:
 1.0 * ZIZZ
+ 1.0 * XIXI 

Custom operator with multiple operators:
 SummedOp([
  1.0*p[0] * ZIZZ,
  1.0*p[1] * XIXI
]) 



Finally, operators can be combined by adding or multiplying them together:

In [8]:
op = SingleProbability(num_qubits=4) + CustomExpectationOperator(
    num_qubits=4, operator_string="ZIZZ", parameterized=True
)
print(
    "Example for summed operator:\n", op.get_pauli(ParameterVector("p", op.num_parameters)), "\n"
)


op = SummedPaulis(num_qubits=4, full_sum=False) * CustomExpectationOperator(
    num_qubits=4, operator_string="XIYZ", parameterized=True
)
print(
    "Example for multiplied operator:\n",
    op.get_pauli(ParameterVector("p", op.num_parameters)),
    "\n",
)

Example for summed operator:
 SummedOp([
  0.5 * IIII
  + 0.5 * IIIZ,
  1.0*p[0] * ZIZZ
]) 

Example for multiplied operator:
 SummedOp([
  1.0*p[0]*p[2] * XIYZ,
  1.0*p[1]*p[2] * XIYI,
  1.0*p[1]*p[2] * -iXIXZ,
  1.0*p[1]*p[2] * XZYZ,
  1.0*p[1]*p[2] * iYIYZ
]) 



The differentiation of the operator can be achieved by the class ``ExpectationOperatorDerivatives``.
It can be also used to compute the squared form of the operator as shown below.

In [9]:
op = IsingHamiltonian(num_qubits=4, Z="S")
op_derivatives = ExpectationOperatorDerivatives(op)
print("Example Operator:\n", op.get_pauli(ParameterVector("p", op.num_parameters)), "\n")

# Calculates the first order derivative with respect to the parameters (result is list that evaluates to the gradient)
print(
    "First order derivative with respect to the parameters:\n",
    op_derivatives.get_derivate("dop"),
    "\n",
)

# Gets parameters that are used in the dervaitve:
param = op_derivatives.get_parameter_vector()
print(
    "Differentiation with respect to the parameter p[1] of the Z term:\n",
    op_derivatives.get_derivate((param[1],)),
    "\n",
)

# Gets the squared operator (e.g. for variance evaluation)
print("Squared operator: \n", op_derivatives.get_operator_squared())

Example Operator:
 SummedOp([
  1.0*p[0] * IIII,
  1.0*p[1] * IIIZ,
  1.0*p[1] * IIZI,
  1.0*p[1] * IZII,
  1.0*p[1] * ZIII,
  1.0*p[2] * IIZZ,
  1.0*p[3] * IZIZ,
  1.0*p[4] * IZZI,
  1.0*p[5] * ZIIZ,
  1.0*p[6] * ZIZI,
  1.0*p[7] * ZZII
]) 

First order derivative with respect to the parameters:
 ListOp([
  OperatorMeasurement(1.0 * IIII),
  OperatorMeasurement(SummedOp([
    1.0 * IIIZ,
    1.0 * IIZI,
    1.0 * IZII,
    1.0 * ZIII
  ])),
  OperatorMeasurement(1.0 * IIZZ),
  OperatorMeasurement(1.0 * IZIZ),
  OperatorMeasurement(1.0 * IZZI),
  OperatorMeasurement(1.0 * ZIIZ),
  OperatorMeasurement(1.0 * ZIZI),
  OperatorMeasurement(1.0 * ZZII)
]) 

Differentiation with respect to the parameter p[1] of the Z term:
 OperatorMeasurement(SummedOp([
  1.0 * IIIZ,
  1.0 * IIZI,
  1.0 * IZII,
  1.0 * ZIII
])) 

Squared operator: 
 OperatorMeasurement(SummedOp([
  1.0*p_op[0]**2 + 4.0*p_op[1]**2 + 1.0*p_op[2]**2 + 1.0*p_op[3]**2 + 1.0*p_op[4]**2 + 1.0*p_op[5]**2 + 1.0*p_op[6]**2 + 1.0*p_op[